In [1]:
# -*- coding: utf-8 -*-

import re
import pandas as pd 
import numpy as np
import math
import random
import tsplib95
from itertools import repeat



#--------------------------------------------------------
def binary_to_dic(path,dimension):
    _matrix = []
    _gr = dict()
    _rank = 0
    for i in range(0,dimension):
        row_ = []
        for j in range(i+1,dimension):
            if int(path[_rank]) == 1:
                row_.append(str(j))
            _rank = _rank + 1
        _matrix.append(row_)   
    _gr =  { str(i) : _matrix[i] for i in range(0, len(_matrix) ) }
    for i in range(0,dimension):
        for element in _gr[str(i)]:
            if str(i) not in _gr[str(element)] :
                _gr[str(element)].append(str(i))
    return _gr
#--------------------------------------------------------
def is_spanning_tree(graph,dimension):
    visited = [] 
    for i in range(0,dimension):
        #print("we start in {}".format(i))
        if dfs(visited, graph,str(i),1,str(i)) == 1 :
            return 1
    return 0  
#--------------------------------------------------------
""" inspired : https://www.educative.io/edpresso/how-to-implement-depth-first-search-in-python """
def dfs(visited, graph, node,_f,_start):
    if node not in visited:
        if _f == 0:
            visited.append(node)
        if len(visited) ==  len(graph):
            if node == _start:
                #print("it s ok : {} ".format(visited))
                return 1
        for neighbour in graph[node]:
            if dfs(visited.copy(), graph, neighbour,0,_start) == 1:
                return 1
    return 0
#--------------------------------------------------------
""" def get_gray_codes(n): is from https://www.sanfoundry.com/python-program-generate-gray-codes-using-recursion/"""
def get_gray_codes(n):
    if n == 0:
        return ['']
    first_half = get_gray_codes(n - 1)
    second_half = first_half.copy()
 
    first_half = ['0' + code for code in first_half]
    second_half = ['1' + code for code in reversed(second_half)]
 
    return first_half + second_half
#--------------------------------------------------------
def generate_random_population(number,dimension):
    random_population = []
    distances = []
    _number_edges = int(dimension*(dimension-1)/2)
    u = get_gray_codes(_number_edges)
    for i in range(0,number):
        _random_choice = random.choice(u)
        if is_valide(_random_choice,dimension) : 
            random_population.append(_random_choice) 
    return random_population
#--------------------------------------------------------
def is_valide(path,dimension):
    _number_edges = 0
    for letter in path :
        if str(letter) == '1':
            _number_edges = _number_edges +1
    if _number_edges < int((dimension))-1 : 
        return 0
    _valide = is_spanning_tree(binary_to_dic(path,dimension),dimension)
    return _valide
#--------------------------------------------------------

def lecture_dataset_tsp(problem,dimension):
    
    pos_x = []
    pos_y = []
    
    for i in range (1,dimension+1) :
        pos_x.append(int(problem.node_coords[i][0]))
        pos_y.append(int(problem.node_coords[i][1]))

   
    df = pd.DataFrame({"pos_x" : pos_x,
                       "pos_y" : pos_y
                           }) 
    
   #print(type(df["pos_x"][0]))
    #print(df)
   #print(type(df["pos_x"][1]))
    #print (pos_x)

    return df

#--------------------------------------------------------
def to_list(df,dimension): 
    list_distances = []  
    for i in range(0,dimension): 
        for j in range(i+1,dimension):
                list_distances.append(calcul_distance(i,j,df)) 
    return list_distances 
#--------------------------------------------------------
def calcul_distance(pos_a, pos_b, df):
    #print(df)
   #print(type(df["pos_x"][0]))
    _ab = int(int(df["pos_x"][pos_a])- int(df["pos_x"][pos_b]))

    _ac = int(int(df["pos_y"][pos_a])- int(df["pos_y"][pos_b]))
    x = int(pow(_ab,2))
    y = int(pow(_ac,2))
    if (x == 0 and y == 0):
    	return np.inf
    else:  
    	return math.sqrt(x+y) 
#--------------------------------------------------------
def distance_from_path(path,distances):
    distance = 0.00
    for i in range(0,len(path)):
        if int(path[i]) == 1 :
            distance = distance + distances[i]
    return distance 
#--------------------------------------------------------
def selection(paths,list_distances,order):
    best_candidates = []
    distances = []
    for i in range(0,len(paths)):
        distances.append(distance_from_path(paths[i],list_distances))
    for j in range(0,order):
        _best_ = np.inf
        _rank_ = 0
        for i in range(0,len(distances)):
            if distances[i] < _best_ :
                _best_ = distances[i]
                _rank_ = i
        if _best_ != np.inf:
            distances.remove(_best_)
            best_candidates.append(paths[_rank_])
    return best_candidates 
#--------------------------------------------------------
def reproduction(random_population,list_distances,dimension,order,_mutation_p):
    best_candidates = []
    _size = len(random_population)
    for i in range(0,_size-1):
        if i%2 == 0 :
            samp = random.sample(range(dimension),int(dimension/2))
            new_path = list(repeat(0, int(dimension*(dimension-1)/2)))
            for j in range(0,dimension*2):
                if j in samp:
                    new_path[j] = str(random_population[i][j])
                else:
                    new_path[j] = str(random_population[i+1][j])

            new_path , viability = mutation_cross_over(new_path,dimension*2,_mutation_p)
            if viability == 1 :
                new_path_str = ""
                for number in new_path:
                    new_path_str = new_path_str + str(number)
                random_population.append(new_path_str)

    return random_population
#--------------------------------------------------------
def mutation_cross_over(new_path,dimension,_mutation_p):
    _p_muta = random.random()
    if _p_muta > _mutation_p:
        _rank = int(random.random()*10)%dimension
        if new_path[_rank] == 0:
            new_path[_rank] = 1
        else :
            new_path[_rank] = 0
    return new_path , is_valide(new_path,int(dimension/2))
#--------------------------------------------------------
def round(random_population,list_distances,order,dimension):
    _mutation_p = 0.98
    for i in reversed(range(1,order)):
        if i%2 == 0 :
            random_population = selection(random_population,list_distances,i)
            random_population = reproduction(random_population,list_distances,dimension,i,_mutation_p)
    random_population = selection(random_population,list_distances,1)
    show_result(random_population[0],list_distances,dimension)
#--------------------------------------------------------
def init_algo(problem,size_population,dimension):
    list_distances = to_list(lecture_dataset_tsp(problem,dimension),dimension)
    random_population = generate_random_population(size_population,dimension)
    return random_population, list_distances
#--------------------------------------------------------
def show_result(path,distances,dimension):
    print("result : ")
    _edge_rank = 0
    print(" {} dist : {}".format(path,distance_from_path(path,distances)))   
#--------------------------------------------------------
def main():
    #**************************************************
    #file_name = "ulysses16.tsp.txt"
    problem = tsplib95.load('ulysses16.tsp.txt')
    # problem = tsplib95.load('a280t.tsp.txt')
    #problem = tsplib95.load('att48.tsp.txt')
    dimension = 6
    #dimension = problem.dimension
    size_population = 10
    order = 2
    #**************************************************
    random_population, list_distances = init_algo(problem,size_population,dimension)
    round(random_population,list_distances,order,dimension)
#--------------------------------------------------------
main()






result : 
 010110110100011 dist : 65.94804354787277
